# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords 
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import ne_chunk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
# !pip install -U spacy
import spacy
from spacy import displacy
from collections import Counter
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_lg
np.random.seed(42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\angang\AppData\Roaming\nltk_data...
[nltk_data]   Package words is alrea

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df[df.columns[4:]]
# X.head()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    Tokenize text.
    
    INPUT
        text (str): text to be tokenized
    OUTPUT
        tokens (list): list of tokens
    '''
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    
    # Reduce words to their root forms
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w).lower().strip() for w in words]
    
    return tokens

# # Sanity check
# tokenize(X[0])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

# # View parameters in pipeline
# pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, Y_train)

C:\Users\angang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\angang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\angang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\angang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\angang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Make predictions based on the trained model
Y_pred = pipeline.predict(X_test)

In [ ]:
# Scores for each category
for i, col in enumerate(Y_test.columns):
    print(f"\nCategory: {col} -----------------------------------")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

In [ ]:
# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Specify parameters for grid search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 50, 100, 200],
#     'clf__estimator__min_samples_split': [2, 5, 10],
#     'clf__estimator__max_depth': [3, 50, None],
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', n_jobs=12, cv=3)
cv.fit(X_train, Y_train)

In [ ]:
# cv.estimator.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Make predictions based on the trained model
Y_pred = cv.predict(X_test)

In [ ]:
# Scores for each category
for i, col in enumerate(Y_test.columns):
    print(f"\nCategory: {col} -----------------------------------")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

In [ ]:
# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Compare different machine learning algorithms

In [ ]:
# Random Forest
# Build model pipeline
pipeline_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

# Train the model
pipeline_nb.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_nb.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

In [ ]:
# Naive Bayes
# Build model pipeline
pipeline_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB())),
])

# Train the model
pipeline_nb.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_nb.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Multinomial Naive Bayes (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

In [ ]:
# Decision Tree
# Build model pipeline
pipeline_dt = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier())),
])

# Train the model
pipeline_dt.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_dt.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Decision Tree (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Overall, Random Forest performs the best among these algorithms, so Random Forest is chosen for further tuning.

#### Add other features besides TF-IDF

In [ ]:
# Create custom transformer
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            if len(tokenize(sentence))>0:
                pos_tags = nltk.pos_tag(tokenize(sentence))
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                    return 1
        return 0

    # Fit method
    def fit(self, X, y=None):
        return self

    # Transform method
    def transform(self, X):
        X_tagged = pd.Series(X, name='starting_verb').apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
# # Sanity check
# starting_verb = StartingVerbExtractor()
# starting_verb.transform(X.iloc[0:10])

In [ ]:
# Recognize named entities in a tagged sentence
nlp = spacy.load('en_core_web_lg')

def named_entities(text):
    doc = nlp(text)
    labels = [ent.label_ for ent in doc.ents]
    labels_count = Counter(labels)
    return labels_count

# Sanity check
X_ent = X.apply(named_entities).apply(pd.Series)
X_ent = X_ent.fillna(0)

In [ ]:
# Create custom transformer
nlp = spacy.load('en_core_web_lg')
class NamedEntityExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def named_entities(self, text):
        doc = nlp(text)
        labels = [ent.label_ for ent in doc.ents]
        labels_count = Counter(labels)
        return labels_count

    # Fit method
    def fit(self, X, y=None):
        return self

    # Transform method
    def transform(self, X):
        X_ent = X.apply(self.named_entities).apply(pd.Series)
        X_ent = X_ent.fillna(0)
        return X_ent
    
# # Sanity check
# named_entity = NamedEntityExtractor()
# named_entity.transform(X.iloc[0:10])

In [ ]:
# Build model pipeline
pipeline_rf_new = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor()), # this is a custom transformer
        
        ('named_entity', NamedEntityExtractor()) # this is a custom transformer
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

In [ ]:
# Train the model
pipeline_rf_new.fit(X_train, Y_train)

# Make predictions based on the trained model
Y_pred = pipeline_rf_new.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Overall, the performance is better than using TF-IDF alone. Therefore, I'm going to use the new pipeline to train the model.

#### Tune the model

In [ ]:
# pipeline_rf_new.get_params()

In [ ]:
# Specify parameters for grid search
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 50, 100, 200],
#     'clf__estimator__min_samples_split': [2, 5, 10],
#     'clf__estimator__max_depth': [3, 50, None],
#     'features__transformer_weights': (
#         {'text_pipeline': 1, 'starting_verb': 1, 'named_entity': 1},
#         {'text_pipeline': 0.5, 'starting_verb': 1, 'named_entity': 1},
#         {'text_pipeline': 1, 'starting_verb': 0.5, 'named_entity': 0.5},
#     )
}

cv = GridSearchCV(pipeline_rf_new, param_grid=parameters, scoring='accuracy', n_jobs=12, cv=3)
cv.fit(X_train, Y_train)

In [ ]:
# cv.estimator.get_params()

#### Evaluate model performance

In [ ]:
# Make predictions based on the trained model
Y_pred = cv.predict(X_test)

# Overall scores
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='micro')
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
print("Random Forest (default)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

### 9. Export your model as a pickle file

In [ ]:
# Save the best model to disk
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.